In [117]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# 1. Load dataset and preprocess

In [118]:
import pandas as pd

In [119]:
df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [120]:
df[['Deck', 'Number', 'Side']] = df['Cabin'].str.split('/', expand=True)

In [121]:
df.drop(columns=['PassengerId', 'Cabin', 'Name'], inplace=True)

In [122]:
mc_homeplanet = df['HomePlanet'].mode()[0]
mc_cryosleep = df['CryoSleep'].mode()[0]
mc_destination = df['Destination'].mode()[0]
mean_age = df['Age'].mean()
mc_vip = df['VIP'].mode()[0]
mean_roomservice = df['RoomService'].mean()
mean_foodcourt = df['FoodCourt'].mean()
mean_shoppingmall = df['ShoppingMall'].mean()
mean_spa = df['Spa'].mean()
mean_vrdeck = df['VRDeck'].mean()
mc_deck = df['Deck'].mode()[0]
mc_side = df['Side'].mode()[0]
mc_number = df['Number'].mode()[0]
df.fillna(value={"HomePlanet": mc_homeplanet, "CryoSleep": mc_cryosleep, "Destination": mc_destination, 
                 'Age': mean_age, 'VIP': mc_vip, 'RoomService': 0, 
                 'FoodCourt': 0, 'ShoppingMall': 0,
                'Spa': 0, 'VRDeck': 0, 'Deck': mc_deck, 'Side': mc_side, 'Number': mc_number}, inplace=True)

/tmp/ipykernel_24/44276989.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(value={"HomePlanet": mc_homeplanet, "CryoSleep": mc_cryosleep, "Destination": mc_destination,


In [123]:
df = pd.get_dummies(df, columns = ['HomePlanet', 'Destination', 'Deck', 'Side'], drop_first=True)
#df = df.dropna()
df = df.astype(int)

In [124]:
y = df['Transported']
X = df.drop(columns=['Transported'])

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_train[0:5]

array([[0.00000000e+00, 3.54430380e-01, 0.00000000e+00, 0.00000000e+00,
        1.98391228e-03, 0.00000000e+00, 3.53219901e-02, 0.00000000e+00,
        4.32946146e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00],
       [0.00000000e+00, 2.15189873e-01, 0.00000000e+00, 0.00000000e+00,
        4.31050031e-02, 1.31959816e-03, 0.00000000e+00, 0.00000000e+00,
        3.03590285e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.00000000e+00, 3.54430380e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.73706441e-01, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.0000

In [126]:
def process_data(df):
    df[['Deck', 'Number', 'Side']] = df['Cabin'].str.split('/', expand=True)
    df.drop(columns=['PassengerId', 'Cabin', 'Name'], inplace=True)
    df.fillna(value={"HomePlanet": mc_homeplanet, "CryoSleep": mc_cryosleep, "Destination": mc_destination, 
                 'Age': mean_age, 'VIP': mc_vip, 'RoomService': 0, 
                 'FoodCourt': 0, 'ShoppingMall': 0,
                'Spa': 0, 'VRDeck': 0, 'Deck': mc_deck, 'Side': mc_side, 'Number': mc_number}, inplace=True)
    #df = df.dropna()
    df = pd.get_dummies(df, columns = ['HomePlanet', 'Destination', 'Deck', 'Side'], drop_first=True)
    df = df.astype(int)
    df = scaler.transform(df)
    return df

In [127]:
X_test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test_passengerID = X_test['PassengerId']
X_test = process_data(X_test)

/tmp/ipykernel_24/3595009695.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(value={"HomePlanet": mc_homeplanet, "CryoSleep": mc_cryosleep, "Destination": mc_destination,


# 2. Evaluate on LogisticRegression Classifier

In [128]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression

param_grid_logreg = {'C': [0.01, 0.02, 0.03, 0.1, 1, 10, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}

grid_logreg = GridSearchCV(LogisticRegression(solver='liblinear'), param_grid_logreg, cv=5)
grid_logreg.fit(X_train, y_train)

print("Best C:", grid_logreg.best_params_)
print("Best Accuracy:", grid_logreg.best_score_)
#Best C: {'C': 400}
#Best Accuracy: 0.79522749018614

best_model = grid_logreg.best_estimator_
y_pred = best_model.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_pred))
#Accuracy: 0.7826336975273146

y_test = grid_logreg.best_estimator_.predict(X_test)


'\nfrom sklearn.linear_model import LogisticRegression\n\nparam_grid_logreg = {\'C\': [0.01, 0.02, 0.03, 0.1, 1, 10, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}\n\ngrid_logreg = GridSearchCV(LogisticRegression(solver=\'liblinear\'), param_grid_logreg, cv=5)\ngrid_logreg.fit(X_train, y_train)\n\nprint("Best C:", grid_logreg.best_params_)\nprint("Best Accuracy:", grid_logreg.best_score_)\n#Best C: {\'C\': 400}\n#Best Accuracy: 0.79522749018614\n'

# 3. Evaluate on DecisionTree Classifier

In [131]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(max_depth=3, max_leaf_nodes=4)
dt_clf.fit(X_train, y_train)

path = dt_clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas  # Array of effective alphas
#ccp_alphas = ccp_alphas[:-1]

param_grid_dt = {
    'ccp_alpha': ccp_alphas,
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}

# 5. Use GridSearchCV
grid_search_dt = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid_dt,
    cv=5,
    scoring='accuracy',
    verbose=1
)

# 6. Fit the model
grid_search_dt.fit(X_train, y_train)
print("Best Parameters:", grid_search_dt.best_params_)
print("Best Accuracy:", grid_search_dt.best_score_)
#Best Parameters: {'ccp_alpha': 0.0, 'max_depth': 10, 'min_samples_split': 5}
#Best Accuracy: 0.7729386756590414

best_model_dt = grid_search_dt.best_estimator_
y_pred = best_model_dt.predict(X_val)

print("Test Accuracy:", accuracy_score(y_val, y_pred))
#Test Accuracy: 0.7619321449108684

y_test = best_model_dt.predict(X_test)


DecisionTreeClassifier(max_depth=3, max_leaf_nodes=4)

# 4. Evaluate on AdaBoost Classifier

In [135]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

base_estimator = DecisionTreeClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200, 300],               # Number of weak learners
    'learning_rate': [0.01, 0.1, 0.5, 1.0],           # Shrinking factor
    'estimator__max_depth': [1, 2, 3]       # Maximum depth of decision trees
}

adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=42)

grid_search_adaboost = GridSearchCV(
    estimator=adaboost,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all available processors
)
grid_search_adaboost.fit(X_train, y_train)

print("Best Parameters:", grid_search_adaboost.best_params_)
print("Best Accuracy:", grid_search_adaboost.best_score_)

#Best Parameters: {'estimator__max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 100}
#Best Accuracy: 0.8098937155092605

best_model = grid_search_adaboost.best_estimator_
y_pred = best_model.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_pred))
#Accuracy: 0.8016101207590569

y_test = best_model.predict(X_test)


'\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.ensemble import AdaBoostClassifier\n\nbase_estimator = DecisionTreeClassifier(random_state=42)\n\nparam_grid = {\n    \'n_estimators\': [50, 100, 200, 300],               # Number of weak learners\n    \'learning_rate\': [0.01, 0.1, 0.5, 1.0],           # Shrinking factor\n    \'estimator__max_depth\': [1, 2, 3]       # Maximum depth of decision trees\n}\n\nadaboost = AdaBoostClassifier(estimator=base_estimator, random_state=42)\n\ngrid_search_adaboost = GridSearchCV(\n    estimator=adaboost,\n    param_grid=param_grid,\n    scoring=\'accuracy\',\n    cv=5,  # 5-fold cross-validation\n    verbose=1,\n    n_jobs=-1  # Use all available processors\n)\ngrid_search_adaboost.fit(X_train, y_train)\n\nprint("Best Parameters:", grid_search_adaboost.best_params_)\nprint("Best Accuracy:", grid_search_adaboost.best_score_)\n\n#Best Parameters: {\'estimator__max_depth\': 3, \'learning_rate\': 0.1, \'n_estimators\': 100}\n#Best Accura

# 5. Evaluate on Random Forest Classifier

In [138]:
from sklearn.ensemble import RandomForestClassifier

param_grid_rf = {
    'n_estimators': [150, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# Best parameters and score
print(f"Best Parameters: {grid_search_rf.best_params_}")
print(f"Best Score: {grid_search_rf.best_score_:.2f}")
#Best Parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
#Best Score: 0.81

best_model = grid_search_rf.best_estimator_
y_pred = best_model.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_pred))
#Accuracy: 0.7947096032202415

y_test = best_model.predict(X_test)


'\nfrom sklearn.ensemble import RandomForestClassifier\n\nparam_grid_rf = {\n    \'n_estimators\': [150, 200, 300],\n    \'max_depth\': [None, 10, 20, 30],\n    \'min_samples_split\': [2, 5, 10],\n    \'min_samples_leaf\': [1, 2, 4],\n    \'max_features\': [\'sqrt\', \'log2\']\n}\n\nrf = RandomForestClassifier(random_state=42)\ngrid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring=\'accuracy\', n_jobs=-1)\ngrid_search_rf.fit(X_train, y_train)\n\n# Best parameters and score\nprint(f"Best Parameters: {grid_search_rf.best_params_}")\nprint(f"Best Score: {grid_search_rf.best_score_:.2f}")\n#Best Parameters: {\'max_depth\': 10, \'max_features\': \'sqrt\', \'min_samples_leaf\': 2, \'min_samples_split\': 5, \'n_estimators\': 200}\n#Best Score: 0.81\n'

# 6. Evaluate on SVM Classifier

In [141]:
from sklearn.svm import SVC

C_range = np.logspace(-5, 5, base=2, num=11)  # 2^-5 to 2^5
gamma_range = np.logspace(-5, 5, base=2, num=11)  # 2^-5 to 2^5

# Create the parameter grid
nlsvm = SVC(kernel="rbf")
grid_param = {
    'C': C_range,
    'gamma': gamma_range
}

# Create the GridSearchCV object with K=3 folds
grid = GridSearchCV(nlsvm, grid_param, cv=3, scoring='accuracy')

# Fit the model on the training data
grid.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)
#Best parameters: {'C': 32.0, 'gamma': 0.25}
#Best cross-validation score: 0.8014092608570608

# Evaluate the best model on the test set
best_model = grid.best_estimator_

y_pred = best_model.predict(X_val)

#print("Accuracy:", accuracy_score(y_val, y_pred))
#Accuracy: 0.78953421506613

y_test = best_model.predict(X_test)


'\nfrom sklearn.svm import SVC\n\nC_range = np.logspace(-5, 5, base=2, num=11)  # 2^-5 to 2^5\ngamma_range = np.logspace(-5, 5, base=2, num=11)  # 2^-5 to 2^5\n\n# Create the parameter grid\nnlsvm = SVC(kernel="rbf")\ngrid_param = {\n    \'C\': C_range,\n    \'gamma\': gamma_range\n}\n\n# Create the GridSearchCV object with K=3 folds\ngrid = GridSearchCV(nlsvm, grid_param, cv=3, scoring=\'accuracy\')\n\n# Fit the model on the training data\ngrid.fit(X_train, y_train)\n\n# Print the best parameters and the corresponding score\nprint("Best parameters:", grid.best_params_)\nprint("Best cross-validation score:", grid.best_score_)\n#Best parameters: {\'C\': 32.0, \'gamma\': 0.25}\n#Best cross-validation score: 0.8014092608570608\n\n# Evaluate the best model on the test set\nbest_model = grid.best_estimator_\n'

# 7. Evaluate on XGBoost Classifier

In [144]:
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],      # Number of boosting rounds
    'max_depth': [3, 5, 7],             # Maximum depth of a tree
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate (eta)
    'subsample': [0.8, 1.0],            # Subsample ratio of the training data
    'colsample_bytree': [0.8, 1.0],     # Subsample ratio of columns
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',    # Metric to optimize
    cv=3,                  # Cross-validation folds
    verbose=1,             # Print progress
    n_jobs=-1              # Use all available cores
)

# Perform grid search
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Test Accuracy:", accuracy)
#Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
#Best Cross-Validation Accuracy: 0.811331607707794
#Test Accuracy: 0.7975848188614146

y_test = best_model.predict(X_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
Best Cross-Validation Accuracy: 0.8119062420803831
Test Accuracy: 0.7987349051178838


In [146]:
import csv

y_test_TF = [False if v == 0 else True for v in y_test]
test_res_df = pd.DataFrame({'PassengerId': test_passengerID, 'Transported': y_test_TF})
test_res_df.to_csv('/kaggle/working/submission.csv', index=False)

# 8. Visualize the results

In [147]:
'''
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Data
data = {
    "Classifier": ["Logistic Regression", "Decision Tree", "AdaBoost", "Random Forest", "SVM"],
    "Train": [0.79522749018614, 0.7729386756590414, 0.8098937155092605, 0.81, 0.80141],
    "Validation": [0.7826336975273146, 0.7619321449108684, 0.8016101207590569, 0.7947096032202415, 0.801409260857060],
    "Test": [0.79167, 0.76876, 0.79845, 0.80079, 0.80266],
}

df = pd.DataFrame(data)

# Visualization
x = np.arange(len(df["Classifier"]))  # Label locations
width = 0.25  # Bar width

fig, ax = plt.subplots(figsize=(10, 6))

# Bars
ax.bar(x - width, df["Train"], width, label="Train", color="skyblue")
ax.bar(x, df["Validation"], width, label="Validation", color="orange")
ax.bar(x + width, df["Test"], width, label="Test", color="green")

# Labels and Title
ax.set_xlabel("Classifiers")
ax.set_ylabel("Accuracy")
ax.set_title("Classifier Performance Comparison")
ax.set_xticks(x)
ax.set_xticklabels(df["Classifier"], rotation=45, ha="right")
ax.legend()

plt.tight_layout()
plt.show()
'''

'\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport numpy as np\n\n# Data\ndata = {\n    "Classifier": ["Logistic Regression", "Decision Tree", "AdaBoost", "Random Forest", "SVM"],\n    "Train": [0.79522749018614, 0.7729386756590414, 0.8098937155092605, 0.81, 0.80141],\n    "Validation": [0.7826336975273146, 0.7619321449108684, 0.8016101207590569, 0.7947096032202415, 0.801409260857060],\n    "Test": [0.79167, 0.76876, 0.79845, 0.80079, 0.80266],\n}\n\ndf = pd.DataFrame(data)\n\n# Visualization\nx = np.arange(len(df["Classifier"]))  # Label locations\nwidth = 0.25  # Bar width\n\nfig, ax = plt.subplots(figsize=(10, 6))\n\n# Bars\nax.bar(x - width, df["Train"], width, label="Train", color="skyblue")\nax.bar(x, df["Validation"], width, label="Validation", color="orange")\nax.bar(x + width, df["Test"], width, label="Test", color="green")\n\n# Labels and Title\nax.set_xlabel("Classifiers")\nax.set_ylabel("Accuracy")\nax.set_title("Classifier Performance Comparison")\nax.set_xtic